In [1]:
%pylab inline
import numpy as np
import scipy as sc
import pandas as pd
from ABC_algorithm import ABC

import statsmodels as sm
from sklearn.linear_model import LinearRegression

Populating the interactive namespace from numpy and matplotlib


In [2]:
######
# set up for the normal ABC example
######

prior_mean = -4.0
prior_sd = 3
likelihood_sd = 1

def NormalPriorSampler(n):
    return np.random.normal(loc=prior_mean, scale=prior_sd, size=n)

def NormalLiklihoodSimulator(n, param):
    #unknown mean
    return np.random.normal(loc=param, scale=likelihood_sd, size=n)
    
def NormalSummary(data):
    return np.mean(data, axis=0)

data = np.random.normal(loc=0,scale=likelihood_sd,size=5)

post_sd = np.power((1.0/np.power(prior_sd,2) + len(data)/np.power(likelihood_sd,2)),-1/2)
post_mean = (prior_mean/np.power(prior_sd,2) + np.sum(data)/(np.power(likelihood_sd,2)))/(1/np.power(prior_sd,2) + len(data)/np.power(likelihood_sd,2))
print post_mean, "is posterior mean"
print post_sd, "is posterior std dev"

df = ABC(NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, 0.5, data, 1000)

0.472376589414 is posterior mean
0.195652173913 is posterior std dev


## A function for post processing ABC algorithm output

In [3]:
def EpanechnikovKernel(t,delta,c=1):
    if t<=delta:
        return c*(1/delta)*(1-(t/delta)**2)
    else:
        return 0

def post_process(abc_df, q=0.5):
    """
    Given a dataframe output to ABC, return dataframe with accepted thetas transformed in column "theta_star"
    """
    # take accpted values output from ABC
    df_accepted = abc_df[abc_df['accept'] == 1]

    df_accepted.statistics_diff = df_accepted.statistics - NormalSummary(data)
    df_accepted.statistics_diff_abs = df_accepted.statistics_diff.apply(lambda x: abs(x))
    
    #df_accepted.statistics_diff_abs.hist(bins=100)
    quantile = df_accepted.statistics_diff_abs.quantile(q)
    #plt.show()

    #create column with kernal transform
    df_accepted.kernel = df_accepted.statistics_diff.apply(lambda x: EpanechnikovKernel(abs(x), delta=quantile))

    #print df_accepted.kernel[df_accepted.kernel > 0.0]

    mod = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
    X = np.array(df_accepted.statistics_diff)
    X.shape = (shape(X)[0],1)
    y = np.array(df_accepted.theta)
    y.shape = (shape(y)[0],1)
    weights = np.array(df_accepted.kernel)
    
    res = mod.fit(X, y, sample_weight=weights)
    #alpha = res.intercept_[0]
    beta = res.coef_[0][0]
    print beta
    beta_stats_diff = beta*df_accepted.statistics_diff
    beta_stats_diff_array = np.array(beta_stats_diff)
    beta_stats_diff_array.shape = (shape(beta_stats_diff_array)[0],1)
    theta_star = np.subtract(y, beta_stats_diff_array)
    theta_star = np.reshape(theta_star,83)
    print np.shape(theta_star)
    df_accepted['theta_star'] = theta_star
    
    return df_accepted